In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense
from tensorflow.keras.optimizers import Adam
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
train_data_generator = ImageDataGenerator(rescale=1./255,rotation_range=0.2,shear_range=0.2,zoom_range=0.2
                                               ,width_shift_range=0.2,height_shift_range=0.2,validation_split=0.2)

In [3]:
batch_size=8

In [4]:
Train_data=train_data_generator.flow_from_directory(r'C:\Users\vanshika kapur\Downloads\drowsiness detection\Prepared_data\Train dataset',
                                                   target_size=(80,80),batch_size=8,class_mode='categorical',subset='training' )

validation_data=train_data_generator.flow_from_directory(r'C:\Users\vanshika kapur\Downloads\drowsiness detection\Prepared_data\Train dataset',
                                                   target_size=(80,80),batch_size=8,class_mode='categorical',subset='validation' )

Found 64669 images belonging to 2 classes.
Found 16166 images belonging to 2 classes.


In [5]:
Test_data_generator=ImageDataGenerator(rescale=1./255)
Test_data=Test_data_generator.flow_from_directory(r'C:\Users\vanshika kapur\Downloads\drowsiness detection\Prepared_data\Test dataset',
                                                  target_size=(80,80),batch_size=8,class_mode='categorical')

Found 4063 images belonging to 2 classes.


In [6]:
Base_model=InceptionV3(include_top=False,weights='imagenet',input_tensor=Input(shape=(80,80,3),batch_size=8))
Head_model=Base_model.output
Head_model=Flatten()(Head_model)
Head_model=Dense(64,activation='relu')(Head_model)#takes only the important data,puts another layer of 64 neuron
Head_model=Dropout(0.5)(Head_model) # prevent overfitting 
Head_model=Dense(2,activation='softmax')(Head_model)#because its a categorical data,it puts another layer of 2 neuron

model=Model(inputs=Base_model.input,outputs=Head_model)
for layer in Base_model.layers:  #it will fuse the in between layers it will take the knowledge of inception layer and train the other layers
    layer.trainable=False

In [7]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (8, 80, 80, 3)            │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (8, 39, 39, 32)           │             864 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (8, 39, 39, 32)           │              96 │ conv2d[0][0]               │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation (Activation)       │ (8, 39, 39, 32)           │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (8, 37, 37, 32)           │           9,216 │ activation[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (8, 37, 37, 32)           │              96 │ conv2d_1[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_1 (Activation)     │ (8, 37, 37, 32)           │               0 │ batch_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (8, 37, 37, 64)           │          18,432 │ activation_1[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (8, 37, 37, 64)           │             192 │ conv2d_2[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_2 (Activation)     │ (8, 37, 37, 64)           │               0 │ batch_normalization_2[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (8, 18, 18, 64)           │               0 │ activation_2[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (8, 18, 18, 80)           │           5,120 │ max_pooling2d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_3         │ (8, 18, 18, 80)           │             240 │ conv2d_3[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_3 (Activation)     │ (8, 18, 18, 80)           │               0 │ batch_normalization_3[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_4 (Conv2D)             │ (8, 16, 16, 192)          │         138,240 │ activation_3[0][0]         │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 21,934,050 (83.67 MB)

 Trainable params: 131,266 (512.76 KB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [9]:

checkpoint = ModelCheckpoint(r'C:\Users\vanshika kapur\Downloads\drowsiness detection\models.keras',
                             monitor='val_loss', save_best_only=True, verbose=3)
earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor='val_loss', verbose=3)

callbacks = [checkpoint, earlystop, learning_rate]


In [10]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x=Train_data,
          steps_per_epoch=Train_data.samples // batch_size,
          validation_data=validation_data,  
          validation_steps=validation_data.samples// batch_size,
          callbacks=callbacks,
          epochs=5)

Epoch 1/5


C:\Users\vanshika kapur\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8083/8083 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.9141 - loss: 0.2238
Epoch 1: val_loss improved from inf to 0.26982, saving model to C:\Users\vanshika kapur\Downloads\drowsiness detection\models.keras
8083/8083 ━━━━━━━━━━━━━━━━━━━━ 1187s 143ms/step - accuracy: 0.9141 - loss: 0.2238 - val_accuracy: 0.8889 - val_loss: 0.2698 - learning_rate: 0.0010
Epoch 2/5
   1/8083 ━━━━━━━━━━━━━━━━━━━━ 12:30 93ms/step - accuracy: 1.0000 - loss: 0.0787

C:\Users\vanshika kapur\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)



Epoch 2: val_loss improved from 0.26982 to 0.21860, saving model to C:\Users\vanshika kapur\Downloads\drowsiness detection\models.keras
8083/8083 ━━━━━━━━━━━━━━━━━━━━ 3s 398us/step - accuracy: 1.0000 - loss: 0.0787 - val_accuracy: 0.8333 - val_loss: 0.2186 - learning_rate: 0.0010
Epoch 3/5
8083/8083 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.9342 - loss: 0.1686
Epoch 3: val_loss did not improve from 0.21860
8083/8083 ━━━━━━━━━━━━━━━━━━━━ 1167s 144ms/step - accuracy: 0.9342 - loss: 0.1686 - val_accuracy: 0.9032 - val_loss: 0.2359 - learning_rate: 0.0010
Epoch 4/5
   1/8083 ━━━━━━━━━━━━━━━━━━━━ 12:51 95ms/step - accuracy: 1.0000 - loss: 0.0296
Epoch 4: val_loss did not improve from 0.21860
8083/8083 ━━━━━━━━━━━━━━━━━━━━ 0s 29us/step - accuracy: 1.0000 - loss: 0.0296 - val_accuracy: 0.8333 - val_loss: 0.3667 - learning_rate: 0.0010
Epoch 5/5
8083/8083 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.9388 - loss: 0.1577
Epoch 5: val_loss did not improve from 0.21860
8083/8083 ━━━━━━